In [3]:
# Install required packages
!pip install pathway bokeh pandas numpy matplotlib seaborn plotly

import pandas as pd
import numpy as np
import pathway as pw
from datetime import datetime, timedelta
import time
import random
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column, row
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Enable Bokeh in Jupyter
output_notebook()

In [21]:
# Load your dataset
print("Loading dataset.csv...")
df = pd.read_csv('dataset.csv')

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"Unique parking spaces: {df['SystemCodeNumber'].nunique()}")

class DataPreprocessor:
    def __init__(self, df):
        self.df = df.copy()
        self.preprocess_data()

    def preprocess_data(self):
        """Clean and prepare your existing dataset for streaming"""

        # Handle DD-MM-YYYY date format from your data
        try:
            self.df['DateTime'] = pd.to_datetime(
                self.df['LastUpdatedDate'] + ' ' + self.df['LastUpdatedTime'],
                format='%d-%m-%Y %H:%M:%S',
                errors='coerce'
            )
        except:
            self.df['DateTime'] = pd.to_datetime(
                self.df['LastUpdatedDate'] + ' ' + self.df['LastUpdatedTime'],
                dayfirst=True,
                errors='coerce'
            )

        # Remove any rows with invalid dates
        initial_count = len(self.df)
        self.df = self.df.dropna(subset=['DateTime'])
        if len(self.df) < initial_count:
            print(f"Removed {initial_count - len(self.df)} rows with invalid dates")

        # Calculate occupancy rate
        self.df['OccupancyRate'] = self.df['Occupancy'] / self.df['Capacity']

        # Create time features
        self.df['Hour'] = self.df['DateTime'].dt.hour
        self.df['DayOfWeek'] = self.df['DateTime'].dt.dayofweek

        # Map traffic conditions to numeric
        traffic_map = {'low': 0, 'average': 1, 'high': 2}
        self.df['TrafficLevel'] = self.df['TrafficConditionNearby'].map(traffic_map)
        self.df['TrafficLevel'] = self.df['TrafficLevel'].fillna(1)  # Default to average

        # Sort by datetime for proper streaming
        self.df = self.df.sort_values(['DateTime', 'SystemCodeNumber']).reset_index(drop=True)

        print(f" Data preprocessing completed!")
        print(f" Processed {len(self.df):,} records")
        print(f" Time range: {self.df['DateTime'].min()} to {self.df['DateTime'].max()}")

    def get_processed_data(self):
        return self.df

# Process your data
preprocessor = DataPreprocessor(df)
processed_df = preprocessor.get_processed_data()

# Show sample
print("\n📋 Sample of processed data:")
display(processed_df[['SystemCodeNumber', 'DateTime', 'OccupancyRate',
                     'QueueLength', 'TrafficConditionNearby']].head())


Loading dataset.csv...
Dataset loaded successfully!
Shape: (18368, 12)
Columns: ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']
Unique parking spaces: 14
 Data preprocessing completed!
 Processed 18,368 records
 Time range: 2016-10-04 07:59:00 to 2016-12-19 16:30:00

📋 Sample of processed data:


,SystemCodeNumber,DateTime,OccupancyRate,QueueLength,TrafficConditionNearby
0,BHMBCCMKT01,2016-10-04 07:59:00,0.105719,1,low
1,BHMBCCTHL01,2016-10-04 07:59:00,0.310078,2,low
2,BHMEURBRD01,2016-10-04 07:59:00,0.248936,2,low
3,BHMMBMMBX01,2016-10-04 07:59:00,0.384279,2,low
4,BHMNCPHST01,2016-10-04 07:59:00,0.197500,2,low


In [22]:
# Define Pathway schema for your parking data
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    DateTime: str
    Capacity: int
    Occupancy: int
    OccupancyRate: float
    VehicleType: str
    TrafficConditionNearby: str
    TrafficLevel: int
    QueueLength: int
    IsSpecialDay: int
    Latitude: float
    Longitude: float

print("Pathway schema defined for parking data")


Pathway schema defined for parking data


In [23]:
class DynamicPricingEngine:
    def __init__(self, base_price=10.0):
        self.base_price = base_price
        self.vehicle_weights = {
            'car': 1.0,
            'truck': 1.5,
            'bike': 0.7,
            'cycle': 0.5
        }
        self.price_history = {}

    def calculate_baseline_price(self, occupancy_rate, current_price=None):
        """Model 1: Baseline Linear Model"""
        if current_price is None:
            current_price = self.base_price

        alpha = 5.0  # Price sensitivity parameter
        price_adjustment = alpha * occupancy_rate
        new_price = current_price + price_adjustment

        # Apply bounds (50% to 200% of base price)
        return max(self.base_price * 0.5, min(new_price, self.base_price * 2.0))

    def calculate_demand_price(self, space_code, occupancy_rate, queue_length,
                             traffic_level, is_special_day, vehicle_type):
        """Model 2: Advanced Demand-Based Model"""

        # Get vehicle weight
        vehicle_weight = self.vehicle_weights.get(vehicle_type, 1.0)

        # Calculate demand components
        occupancy_component = 0.4 * occupancy_rate
        queue_component = 0.3 * min(queue_length / 10.0, 1.0)
        traffic_component = 0.2 * (traffic_level / 2.0)
        special_day_component = 0.1 * is_special_day

        # Total demand score
        demand_score = (occupancy_component + queue_component +
                       traffic_component + special_day_component)

        # Apply vehicle type multiplier
        adjusted_demand = demand_score * vehicle_weight

        # Calculate price
        price_multiplier = 1 + (adjusted_demand * 0.8)
        new_price = self.base_price * price_multiplier

        # Apply bounds and smoothing
        bounded_price = max(self.base_price * 0.5, min(new_price, self.base_price * 2.0))

        # Smooth price transitions
        if space_code in self.price_history and self.price_history[space_code]:
            last_price = self.price_history[space_code][-1]
            max_change = last_price * 0.1  # Max 10% change
            if abs(bounded_price - last_price) > max_change:
                bounded_price = last_price + (max_change if bounded_price > last_price else -max_change)

        return round(bounded_price, 2)

    def update_price_history(self, space_code, price):
        """Update price history for a parking space"""
        if space_code not in self.price_history:
            self.price_history[space_code] = []

        self.price_history[space_code].append(price)

        # Keep only last 50 prices
        if len(self.price_history[space_code]) > 50:
            self.price_history[space_code] = self.price_history[space_code][-50:]

# Initialize pricing engine
pricing_engine = DynamicPricingEngine()
print("Dynamic Pricing Engine initialized")


Dynamic Pricing Engine initialized


In [24]:
class PathwayStreamingProcessor:
    def __init__(self, processed_df, pricing_engine):
        self.df = processed_df
        self.pricing_engine = pricing_engine
        self.results = []
        self.current_index = 0

    def create_streaming_data(self, batch_size=14, delay_seconds=2):
        """
        Simulate real-time data streaming with delay
        Processes data in batches (one timestamp at a time)
        """
        print("Starting Pathway real-time simulation...")
        print(f"Total data points: {len(self.df)}")

        # Get unique timestamps for streaming
        unique_timestamps = sorted(self.df['DateTime'].unique())
        print(f" Processing {len(unique_timestamps)} timestamps")

        for i, timestamp in enumerate(unique_timestamps[:20]):  # Process first 20 timestamps for demo
            # Get data for current timestamp
            current_batch = self.df[self.df['DateTime'] == timestamp].copy()

            if current_batch.empty:
                continue

            print(f"\n Streaming batch {i+1}/{min(20, len(unique_timestamps))}: {timestamp}")
            print(f"   Processing {len(current_batch)} parking spaces")

            # Process each parking space in the batch
            batch_results = []
            for _, row in current_batch.iterrows():
                # Calculate prices using both models
                baseline_price = self.pricing_engine.calculate_baseline_price(
                    row['OccupancyRate']
                )

                demand_price = self.pricing_engine.calculate_demand_price(
                    row['SystemCodeNumber'], row['OccupancyRate'], row['QueueLength'],
                    row['TrafficLevel'], row['IsSpecialDay'], row['VehicleType']
                )

                # Update price history
                self.pricing_engine.update_price_history(row['SystemCodeNumber'], demand_price)

                # Create result record
                result = {
                    'Timestamp': timestamp,
                    'SystemCodeNumber': row['SystemCodeNumber'],
                    'Occupancy': row['Occupancy'],
                    'Capacity': row['Capacity'],
                    'OccupancyRate': row['OccupancyRate'],
                    'QueueLength': row['QueueLength'],
                    'TrafficCondition': row['TrafficConditionNearby'],
                    'VehicleType': row['VehicleType'],
                    'IsSpecialDay': row['IsSpecialDay'],
                    'BaselinePrice': baseline_price,
                    'DemandPrice': demand_price,
                    'FinalPrice': demand_price  # Use demand price as final
                }
                batch_results.append(result)

            # Store batch results
            if batch_results:
                batch_df = pd.DataFrame(batch_results)
                self.results.append(batch_df)

                # Show current pricing status
                avg_price = batch_df['FinalPrice'].mean()
                avg_occupancy = batch_df['OccupancyRate'].mean()
                print(f"    Average price: ${avg_price:.2f}")
                print(f"    Average occupancy: {avg_occupancy:.1%}")

                # Show real-time pricing for each space
                for _, space_data in batch_df.iterrows():
                    print(f"    {space_data['SystemCodeNumber']}: ${space_data['FinalPrice']:.2f} "
                          f"(Occupancy: {space_data['OccupancyRate']:.1%})")

            # Simulate streaming delay
            time.sleep(delay_seconds)

        print(f"\nStreaming simulation completed!")
        return self.get_combined_results()

    def get_combined_results(self):
        """Combine all streaming results"""
        if not self.results:
            return pd.DataFrame()
        return pd.concat(self.results, ignore_index=True)

# Initialize streaming processor
stream_processor = PathwayStreamingProcessor(processed_df, pricing_engine)


In [25]:
def run_realtime_simulation():
    """Run the complete real-time pricing simulation"""

    print("=" * 70)
    print("DYNAMIC PARKING PRICING - REAL-TIME SIMULATION")
    print("=" * 70)

    # Start streaming simulation
    print("\nInitiating Pathway streaming simulation...")

    # Run streaming with your data
    streaming_results = stream_processor.create_streaming_data(
        batch_size=14,      # All parking spaces per batch
        delay_seconds=1     # 1 second delay between timestamps
    )

    if not streaming_results.empty:
        print(f"\n📊 SIMULATION RESULTS")
        print(f"✅ Processed {len(streaming_results)} pricing decisions")
        print(f"🏢 Covered {streaming_results['SystemCodeNumber'].nunique()} parking spaces")
        print(f"🕐 Time span: {streaming_results['Timestamp'].min()} to {streaming_results['Timestamp'].max()}")

        # Pricing statistics
        print(f"\n💰 PRICING ANALYSIS")
        print(f"💵 Price range: ${streaming_results['FinalPrice'].min():.2f} - ${streaming_results['FinalPrice'].max():.2f}")
        print(f"📈 Average price: ${streaming_results['FinalPrice'].mean():.2f}")
        print(f"📊 Price std dev: ${streaming_results['FinalPrice'].std():.2f}")

        # Model comparison
        print(f"\n🔄 MODEL PERFORMANCE")
        print(f"📉 Baseline model avg: ${streaming_results['BaselinePrice'].mean():.2f}")
        print(f"📈 Demand model avg: ${streaming_results['DemandPrice'].mean():.2f}")

        # Occupancy insights
        occupancy_price_corr = streaming_results['OccupancyRate'].corr(streaming_results['FinalPrice'])
        print(f"🔗 Occupancy-Price correlation: {occupancy_price_corr:.3f}")

        return streaming_results
    else:
        print("❌ No results generated")
        return pd.DataFrame()

# Execute the simulation
final_results = run_realtime_simulation()


DYNAMIC PARKING PRICING - REAL-TIME SIMULATION

Initiating Pathway streaming simulation...
Starting Pathway real-time simulation...
Total data points: 18368
 Processing 1445 timestamps

 Streaming batch 1/20: 2016-10-04 07:59:00
   Processing 14 parking spaces
    Average price: $11.28
    Average occupancy: 26.8%
    BHMBCCMKT01: $10.58 (Occupancy: 10.6%)
    BHMBCCTHL01: $11.47 (Occupancy: 31.0%)
    BHMEURBRD01: $11.28 (Occupancy: 24.9%)
    BHMMBMMBX01: $11.71 (Occupancy: 38.4%)
    BHMNCPHST01: $10.78 (Occupancy: 19.8%)
    BHMNCPNST01: $12.12 (Occupancy: 51.3%)
    Broad Street: $11.31 (Occupancy: 25.8%)
    Others-CCCPS105a: $11.61 (Occupancy: 35.3%)
    Others-CCCPS119a: $10.46 (Occupancy: 7.0%)
    Others-CCCPS135a: $11.37 (Occupancy: 27.8%)
    Others-CCCPS202: $10.75 (Occupancy: 18.6%)
    Others-CCCPS8: $11.82 (Occupancy: 33.7%)
    Others-CCCPS98: $11.89 (Occupancy: 18.9%)
    Shopping: $10.75 (Occupancy: 32.0%)

 Streaming batch 2/20: 2016-10-04 08:25:00
   Processing 14 

In [26]:
def create_comprehensive_dashboard(results_df):
    """Create comprehensive real-time Bokeh dashboard"""

    if results_df.empty:
        print("No data for visualization")
        return

    print("📊 Creating comprehensive pricing dashboard...")

    # Prepare data
    spaces = results_df['SystemCodeNumber'].unique()
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
              '#aec7e8', '#ffbb78', '#98df8a', '#ff9896']

    space_colors = {space: colors[i % len(colors)] for i, space in enumerate(spaces)}

    # Plot 1: Real-time pricing trends for all 14 spaces (MEDIUM SIZE)
    price_plot = figure(
        title="Real-Time Parking Prices by Space (All 14 Spaces)",
        x_axis_label='Time',
        y_axis_label='Price ($)',
        width=850,   # Medium width
        height=350,  # Medium height
        x_axis_type='datetime',
        tools="pan,wheel_zoom,box_zoom,reset,save,hover"
    )

    # Add hover tool
    price_plot.hover.tooltips = [
        ('Space', '@space'),
        ('Time', '@x{%H:%M:%S}'),
        ('Price', '$@y{0.00}'),
        ('Occupancy', '@occupancy{0.0%}')
    ]
    price_plot.hover.formatters = {'@x': 'datetime'}

    # Add price lines for each space
    for space in spaces:
        space_data = results_df[results_df['SystemCodeNumber'] == space].sort_values('Timestamp')
        if not space_data.empty:
            source = ColumnDataSource(data=dict(
                x=space_data['Timestamp'],
                y=space_data['FinalPrice'],
                space=[space] * len(space_data),
                occupancy=space_data['OccupancyRate']
            ))

            # Line plot
            price_plot.line('x', 'y', source=source,
                          legend_label=space, color=space_colors[space],
                          line_width=2, alpha=0.8)
            # Circle markers
            price_plot.circle('x', 'y', source=source,
                            color=space_colors[space], size=5, alpha=0.7)

    # Style the legend
    price_plot.legend.location = "top_left"
    price_plot.legend.click_policy = "hide"
    price_plot.legend.label_text_font_size = "9pt"
    price_plot.title.text_font_size = "13pt"

    # Plot 2: Pricing Model Comparison (ESSENTIAL - MEDIUM SIZE)
    comparison_plot = figure(
        title="Pricing Model Comparison - Baseline vs Demand-Based vs Final",
        x_axis_label='Time',
        y_axis_label='Price ($)',
        width=850,   # Medium width
        height=300,  # Medium height
        x_axis_type='datetime',
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )

    # Calculate average prices across all spaces for each model
    time_avg = results_df.groupby('Timestamp').agg({
        'BaselinePrice': 'mean',
        'DemandPrice': 'mean',
        'FinalPrice': 'mean'
    }).reset_index()

    # Add lines for each model
    comparison_plot.line(
        time_avg['Timestamp'], time_avg['BaselinePrice'],
        legend_label='Baseline Model (Linear)', color='red', line_width=3
    )
    comparison_plot.circle(
        time_avg['Timestamp'], time_avg['BaselinePrice'],
        color='red', size=6, alpha=0.7
    )

    comparison_plot.line(
        time_avg['Timestamp'], time_avg['DemandPrice'],
        legend_label='Demand-Based Model', color='green', line_width=3
    )
    comparison_plot.circle(
        time_avg['Timestamp'], time_avg['DemandPrice'],
        color='green', size=6, alpha=0.7
    )

    comparison_plot.line(
        time_avg['Timestamp'], time_avg['FinalPrice'],
        legend_label='Final Optimized Price', color='blue', line_width=3
    )
    comparison_plot.circle(
        time_avg['Timestamp'], time_avg['FinalPrice'],
        color='blue', size=6, alpha=0.7
    )

    # Style the comparison plot
    comparison_plot.legend.location = "top_left"
    comparison_plot.legend.label_text_font_size = "10pt"
    comparison_plot.title.text_font_size = "13pt"

    # Plot 3: Occupancy vs Price Correlation (ESSENTIAL - MEDIUM SIZE)
    occupancy_plot = figure(
        title="Occupancy Rate vs Price Correlation (Pricing Justification)",
        x_axis_label='Occupancy Rate (%)',
        y_axis_label='Price ($)',
        width=850,   # Medium width
        height=300,  # Medium height
        tools="pan,wheel_zoom,box_zoom,reset,save,hover"
    )

    # Add scatter points for all spaces
    for space in spaces:
        space_data = results_df[results_df['SystemCodeNumber'] == space]
        if not space_data.empty:
            source = ColumnDataSource(data=dict(
                x=space_data['OccupancyRate'] * 100,
                y=space_data['FinalPrice'],
                space=[space] * len(space_data),
                queue=space_data['QueueLength']
            ))

            occupancy_plot.circle('x', 'y', source=source,
                                color=space_colors[space], size=7, alpha=0.6,
                                legend_label=space)

    # Add trend line for correlation
    from scipy import stats
    x_all = results_df['OccupancyRate'] * 100
    y_all = results_df['FinalPrice']
    slope, intercept, r_value, p_value, std_err = stats.linregress(x_all, y_all)

    x_trend = np.array([x_all.min(), x_all.max()])
    y_trend = slope * x_trend + intercept

    occupancy_plot.line(x_trend, y_trend,
                       color='red', line_width=3, alpha=0.8,
                       legend_label=f'Correlation Trend (r={r_value:.3f})')

    # Configure hover for occupancy plot
    occupancy_plot.hover.tooltips = [
        ('Space', '@space'),
        ('Occupancy', '@x{0.1}%'),
        ('Price', '$@y{0.00}'),
        ('Queue Length', '@queue')
    ]

    # Style the occupancy plot
    occupancy_plot.legend.location = "top_left"
    occupancy_plot.legend.click_policy = "hide"
    occupancy_plot.legend.label_text_font_size = "9pt"
    occupancy_plot.title.text_font_size = "13pt"

    # Create layout with medium sizing
    layout = column(
        price_plot,
        comparison_plot,
        occupancy_plot,
        sizing_mode='scale_width'
    )

    show(layout)

    # Print summary statistics
    print(f"\n Dashboard created successfully!")
    print(f"Visualized {len(spaces)} parking spaces")
    print(f"Price range: ${results_df['FinalPrice'].min():.2f} - ${results_df['FinalPrice'].max():.2f}")
    print(f"Average price: ${results_df['FinalPrice'].mean():.2f}")
    print(f"Occupancy-price correlation: {r_value:.3f}")
    print(f"Total data points: {len(results_df):,}")

    return layout

# Enhanced function call
if 'final_results' in globals() and not final_results.empty:
    comprehensive_dashboard = create_comprehensive_dashboard(final_results)
else:
    print("Please run the simulation first to generate results data")


📊 Creating comprehensive pricing dashboard...



 Dashboard created successfully!
Visualized 14 parking spaces
Price range: $10.46 - $17.03
Average price: $13.14
Occupancy-price correlation: 0.719
Total data points: 266


In [30]:
def generate_business_insights(results_df):
    """Generate comprehensive business insights"""

    if results_df.empty:
        print("No data for analysis")
        return

    print("=" * 70)
    print(" BUSINESS INSIGHTS & PRICING STRATEGY")
    print("=" * 70)

    # Performance by parking space
    space_performance = results_df.groupby('SystemCodeNumber').agg({
        'FinalPrice': ['mean', 'min', 'max', 'std'],
        'OccupancyRate': ['mean', 'max'],
        'QueueLength': 'mean'
    }).round(2)

    print("\n🏢 PARKING SPACE PERFORMANCE")
    display(space_performance)

    # Peak pricing analysis
    high_occupancy = results_df[results_df['OccupancyRate'] > 0.8]
    if not high_occupancy.empty:
        print(f"\n PEAK DEMAND ANALYSIS (>80% occupancy)")
        print(f"Peak situations: {len(high_occupancy)} instances")
        print(f"Average peak price: ${high_occupancy['FinalPrice'].mean():.2f}")
        print(f"Peak price range: ${high_occupancy['FinalPrice'].min():.2f} - ${high_occupancy['FinalPrice'].max():.2f}")

    # Vehicle type pricing
    vehicle_analysis = results_df.groupby('VehicleType').agg({
        'FinalPrice': 'mean',
        'OccupancyRate': 'mean'
    }).round(2)

    print(f"\n VEHICLE TYPE PRICING")
    display(vehicle_analysis)

    # Queue impact
    if results_df['QueueLength'].max() > 0:
        queue_impact = results_df[results_df['QueueLength'] > 0]
        print(f"\n QUEUE IMPACT ANALYSIS")
        print(f"Situations with queues: {len(queue_impact)} instances")
        if not queue_impact.empty:
            print(f"Average price with queues: ${queue_impact['FinalPrice'].mean():.2f}")
            print(f"Average price without queues: ${results_df[results_df['QueueLength'] == 0]['FinalPrice'].mean():.2f}")

    # Revenue optimization recommendations
    print(f"\n OPTIMIZATION RECOMMENDATIONS")
    recommendations = [
        f" Implement dynamic pricing - potential {((results_df['FinalPrice'].max() - results_df['FinalPrice'].min()) / results_df['FinalPrice'].mean() * 100):.1f}% revenue increase",
        f" Focus on high-demand spaces: {space_performance['FinalPrice']['mean'].sort_values(ascending=False).index[0]}",
        f" Peak pricing during high occupancy periods can increase revenue",
        f" Premium pricing for trucks shows {((vehicle_analysis.loc['truck', 'FinalPrice'] / vehicle_analysis.loc['car', 'FinalPrice'] - 1) * 100):.1f}% uplift" if 'truck' in vehicle_analysis.index and 'car' in vehicle_analysis.index else "🚚 Implement vehicle-specific pricing",
        f" Real-time monitoring reduces customer wait times and optimizes utilization"
    ]

    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")

# Generate insights if we have results
if 'final_results' in globals() and not final_results.empty:
    generate_business_insights(final_results)


 BUSINESS INSIGHTS & PRICING STRATEGY

🏢 PARKING SPACE PERFORMANCE


FinalPrice                     OccupancyRate        \
                       mean    min    max   std          mean   max   
SystemCodeNumber                                                      
BHMBCCMKT01           12.22  10.58  14.74  1.44          0.30  0.47   
BHMBCCTHL01           13.53  11.08  16.07  1.64          0.68  0.86   
BHMEURBRD01           13.35  11.00  16.58  1.65          0.79  0.99   
BHMMBMMBX01           13.97  11.40  17.02  1.67          0.75  0.95   
BHMNCPHST01           12.78  10.78  14.30  1.24          0.52  0.65   
BHMNCPNST01           13.62  12.12  15.79  1.12          0.69  0.79   
Broad Street          13.64  11.31  16.93  1.64          0.78  0.99   
Others-CCCPS105a      13.03  11.16  15.50  1.18          0.60  0.74   
Others-CCCPS119a      11.94  10.46  14.20  1.19          0.17  0.22   
Others-CCCPS135a      13.84  11.35  16.66  1.84          0.68  0.84   
Others-CCCPS202       12.94  10.75  15.25  1.40          0.39  0.48   
Others-CCCPS8         13.49  11.46  17.03  1.74          0.52  0.63   
Others-CCCPS98        12.42  10.81  14.51  1.12          0.34  0.42   
Shopping              13.21  10.75  15.74  1.30          0.63  0.76   

                 QueueLength  
                        mean  
SystemCodeNumber              
BHMBCCMKT01             3.47  
BHMBCCTHL01             3.53  
BHMEURBRD01             3.89  
BHMMBMMBX01             4.00  
BHMNCPHST01             3.89  
BHMNCPNST01             4.11  
Broad Street            4.21  
Others-CCCPS105a        4.53  
Others-CCCPS119a        3.84  
Others-CCCPS135a        5.11  
Others-CCCPS202         4.74  
Others-CCCPS8           4.16  
Others-CCCPS98          4.74  
Shopping                4.47


 PEAK DEMAND ANALYSIS (>80% occupancy)
Peak situations: 49 instances
Average peak price: $14.85
Peak price range: $12.23 - $17.02

 VEHICLE TYPE PRICING


,FinalPrice,OccupancyRate
VehicleType,,
bike,12.54,0.56
car,13.27,0.56
cycle,12.14,0.59
truck,14.10,0.57



 QUEUE IMPACT ANALYSIS
Situations with queues: 266 instances
Average price with queues: $13.14
Average price without queues: $nan

 OPTIMIZATION RECOMMENDATIONS
1.  Implement dynamic pricing - potential 50.0% revenue increase
2.  Focus on high-demand spaces: BHMMBMMBX01
3.  Peak pricing during high occupancy periods can increase revenue
4.  Premium pricing for trucks shows 6.3% uplift
5.  Real-time monitoring reduces customer wait times and optimizes utilization
